# DataFrame Operations

[**Watch the video**](https://panoptotech.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=45909ccc-decb-4a40-95f4-afa8014bbc0f)


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import*

spark = SparkSession.builder.appName('DataFrame Operations').getOrCreate()
sc = spark.sparkContext

In [5]:
# Read a CSV into a dataframe, inferring the schema.

# The decompression of gzip is automatic
dataPath = "../data/Open_Parking_and_Camera_Violations_1M.csv.gz"
fines = spark.read.format("csv")\
  .option("header","true")\
  .option("inferSchema", "true")\
  .load(dataPath)
  
# inferSchema means we will automatically figure out column types 
# at a cost of reading the data more than once

In [6]:
fines.printSchema()

root
 |-- Plate: string (nullable = true)
 |-- State: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- Summons Number: long (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Violation: string (nullable = true)
 |-- Judgment Entry Date: string (nullable = true)
 |-- Fine Amount: double (nullable = true)
 |-- Penalty Amount: double (nullable = true)
 |-- Interest Amount: double (nullable = true)
 |-- Reduction Amount: double (nullable = true)
 |-- Payment Amount: double (nullable = true)
 |-- Amount Due: double (nullable = true)
 |-- Precinct: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Violation Status: string (nullable = true)
 |-- Summons Image: string (nullable = true)



In [7]:
# A schema is a StructType made up of a number of fields, StructFields, that have a name, type, and a Boolean flag which specifies whether that column can contain missing or null values.
fines.schema

StructType(List(StructField(Plate,StringType,true),StructField(State,StringType,true),StructField(License Type,StringType,true),StructField(Summons Number,LongType,true),StructField(Issue Date,StringType,true),StructField(Violation Time,StringType,true),StructField(Violation,StringType,true),StructField(Judgment Entry Date,StringType,true),StructField(Fine Amount,DoubleType,true),StructField(Penalty Amount,DoubleType,true),StructField(Interest Amount,DoubleType,true),StructField(Reduction Amount,DoubleType,true),StructField(Payment Amount,DoubleType,true),StructField(Amount Due,DoubleType,true),StructField(Precinct,IntegerType,true),StructField(County,StringType,true),StructField(Issuing Agency,StringType,true),StructField(Violation Status,StringType,true),StructField(Summons Image,StringType,true)))

In [8]:
fines.show(2)

+-------+-----+------------+--------------+----------+--------------+----------------+-------------------+-----------+--------------+---------------+----------------+--------------+----------+--------+------+-----------------+----------------+--------------------+
|  Plate|State|License Type|Summons Number|Issue Date|Violation Time|       Violation|Judgment Entry Date|Fine Amount|Penalty Amount|Interest Amount|Reduction Amount|Payment Amount|Amount Due|Precinct|County|   Issuing Agency|Violation Status|       Summons Image|
+-------+-----+------------+--------------+----------+--------------+----------------+-------------------+-----------+--------------+---------------+----------------+--------------+----------+--------+------+-----------------+----------------+--------------------+
|1994439|   ME|         PAS|    1438400834|06/05/2018|        07:58A|DETACHED TRAILER|               null|       45.0|          10.0|            0.0|             0.0|           0.0|      55.0|      94|    

In [9]:
fines.select(['Plate', 'Violation']).show(5)

+-------+--------------------+
|  Plate|           Violation|
+-------+--------------------+
|1994439|    DETACHED TRAILER|
|GJK1149|           CROSSWALK|
|HXP4226|REG. STICKER-EXPI...|
|1742308|    DETACHED TRAILER|
| RR2L42|NO STANDING-DAY/T...|
+-------+--------------------+
only showing top 5 rows



Let's see how to manually specify a known schema for a data file, so we can skip the costly "Infer Schema":



In [10]:
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, IntegerType

schema = StructType([StructField('Plate',StringType(), True),
                     StructField('State',StringType(),True),
                     StructField('License Type',StringType()),
                     StructField('Summons Number',IntegerType()),
                     StructField('Issue Date',StringType()),
                     StructField('Violation Time',StringType()),
                     StructField('Violation',StringType()),
                     StructField('Judgment Entry Date',StringType()),
                     StructField('Fine Amount',DoubleType(),True),
                     StructField('Penalty Amount',DoubleType(),True),
                     StructField('Interest Amount',DoubleType(),True),
                     StructField('Reduction Amount',DoubleType(),True),
                     StructField('Payment Amount',DoubleType(),True),
                     StructField('Amount Due',DoubleType(),True),
                     StructField('Precinct',IntegerType(),True),
                     StructField('County',StringType(),True),
                     StructField('Issuing Agency',StringType(),True),
                     StructField('Violation Status',StringType(),True),
                     StructField('Summons Image',StringType(),True)])


fine2 = spark.read.format("csv")\
  .option("header","true")\
  .schema(schema)\
  .load(dataPath)

fine2.printSchema()
#fine2.show(5)

root
 |-- Plate: string (nullable = true)
 |-- State: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- Summons Number: integer (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Violation: string (nullable = true)
 |-- Judgment Entry Date: string (nullable = true)
 |-- Fine Amount: double (nullable = true)
 |-- Penalty Amount: double (nullable = true)
 |-- Interest Amount: double (nullable = true)
 |-- Reduction Amount: double (nullable = true)
 |-- Payment Amount: double (nullable = true)
 |-- Amount Due: double (nullable = true)
 |-- Precinct: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Violation Status: string (nullable = true)
 |-- Summons Image: string (nullable = true)




Now that we've explored the data, let's return to understanding
**transformations** and **actions**.  
Let's create several transformations and then an action. After that we
will inspect exactly what's happening under the hood.

These transformations are simple, first we group by two variables and then compute the average fine.
Then we're going to inner join that to the original dataset on the column State.
Then we'll select the Violation Status from that new dataset.

In [11]:
# a simple grouping
df1 = fines.groupBy(["State","Violation"]).avg("Fine Amount") 

# a simple join and selecting some columns
df2 = df1.join(fines, on=["Violation"], how='inner').select(["Plate","Violation","Violation Status"])

df1.show(10)
df2.show(10)

+-----+--------------------+-----------------+
|State|           Violation| avg(Fine Amount)|
+-----+--------------------+-----------------+
|   NC|OBSTRUCTING DRIVEWAY|94.95652173913044|
|   PA|FAILURE TO STOP A...|             50.0|
|   FL|FRONT OR BACK PLA...|             65.0|
|   MO|NO STANDING-BUS STOP|            115.0|
|   TX|NO STANDING-BUS LANE|            115.0|
|   CT|        TRAFFIC LANE|            115.0|
|   AL|           CROSSWALK|            115.0|
|   MN|    DETACHED TRAILER|             45.0|
|   MS|NO PARKING-DAY/TI...|           60.625|
|   CT|BUS PARKING IN LO...|            102.5|
+-----+--------------------+-----------------+
only showing top 10 rows

+-------+----------------+----------------+
|  Plate|       Violation|Violation Status|
+-------+----------------+----------------+
|1994439|DETACHED TRAILER| HEARING PENDING|
|1994439|DETACHED TRAILER| HEARING PENDING|
|1994439|DETACHED TRAILER| HEARING PENDING|
|1994439|DETACHED TRAILER| HEARING PENDING|
|1994439

In [ ]:
#sort df2 by Violation Status and show only unique Violation Status
df2.sort("Violation Status").select("Violation Status").distinct().show()
#show the fine amount for each violation status
df2.groupBy("Violation Status").avg("Fine Amount").show()
# what is the average fine amount for each violation status for each state?
df2.groupBy(["State","Violation Status"]).avg("Fine Amount").show()

df2.limit(1000).toPandas().memory_usage() # how much memory does this take? returns 800

In [12]:
df2.explain()   

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Plate#279, Violation#76, Violation Status#296]
   +- BroadcastHashJoin [Violation#76], [Violation#285], Inner, BuildLeft, false
      :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]),false), [id=#279]
      :  +- HashAggregate(keys=[State#71, Violation#76], functions=[])
      :     +- Exchange hashpartitioning(State#71, Violation#76, 200), ENSURE_REQUIREMENTS, [id=#276]
      :        +- HashAggregate(keys=[State#71, Violation#76], functions=[])
      :           +- Filter isnotnull(Violation#76)
      :              +- FileScan csv [State#71,Violation#76] Batched: false, DataFilters: [isnotnull(Violation#76)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/data/Open_Parking_and_Camera_Violations_1M.csv.gz], PartitionFilters: [], PushedFilters: [IsNotNull(Violation)], ReadSchema: struct<State:string,Violation:string>
      +- Filter isnotnull(Violation#285)
         +-

In [13]:
# This will execute the plan that Apache Spark built up previously. 
df2.count()

55577292

In [23]:
# we can convert to Pandas IF the data can fit into one node
df2.sample(0.0001).toPandas()

,Plate,Violation,Violation Status
0,51126JU,DOUBLE PARKING,HEARING HELD-GUILTY REDUCTION
1,TUX21W,BUS LANE VIOLATION,None
2,HJYA96,FAIL TO DSPLY MUNI METER RECPT,HEARING HELD-GUILTY
3,8MXL817,NO PARKING-STREET CLEANING,None
4,JMK5878,NO PARKING-STREET CLEANING,None
...,...,...,...
5611,31790ME,NO STANDING-BUS STOP,None
5612,HJZ8615,FRONT OR BACK PLATE MISSING,None
5613,63662JM,DOUBLE PARKING,HEARING HELD-GUILTY REDUCTION
5614,XCHK61,COMML PLATES-UNALTERED VEHICLE,None


# Check yourself
* replace `df2.sample(0.0001).toPandas()` with `df2.toPandas()`  and run the cell.
What happens?
How much memory is needed in this PC to load the full Pandas DF?

hint: `df2.limit(100).toPandas().memory_usage()`